In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset
data = np.load('lab2_dataset.npz')
train_feats = torch.tensor(data['train_feats']) # 44730 examples, 11 frames/example, 40 mfcc bins
test_feats = torch.tensor(data['test_feats']) # 4773 examples, 11 frames/example, 40 mfcc bins
train_labels = torch.tensor(data['train_labels']) # 44730 training data labels 
test_labels = torch.tensor(data['test_labels']) # 4773 testing data labels 
phone_labels = data['phone_labels'] # 48 labels for the 48 classifications

# Set up the dataloaders
train_dataset = torch.utils.data.TensorDataset(train_feats, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(test_feats, test_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)

# Define the model architecture
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # TODO: Fill in the model's layers here
        self.conv1 = nn.Conv2d(1, 4, 3, padding=1)
        self.pool = nn.AvgPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 10, 3, padding=1)
        self.conv3 = nn.Conv2d(10, 16, 3, padding=1)
        self.batchNorm1 = nn.BatchNorm2d(4)
        self.batchNorm2 = nn.BatchNorm2d(10)
        #self.batchNorm2 = nn.BatchNorm1d(48)
        self.fc1 = nn.Linear(10*8, 48)
        #self.fc2 = nn.Linear(96, 48)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # TODO: Fill in the forward pass here
        
        x = torch.unsqueeze(x, dim=1)
        #print(x.shape)
        #print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        #x = self.dropout(x)
        x = self.batchNorm1(x)
        #print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.batchNorm2(x)
        #print(x.shape)
        #x = self.dropout(x)
        x = self.pool(F.relu(self.conv3(x)))
        #print(x.shape)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        #x = self.batchNorm2(x)
        #x = self.fc2
        return x

# Instantiate the model, loss function, and optimizer
model = MyModel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train_network(model, train_loader, criterion, optimizer):
    # TODO: fill in
    numEpochs = 10
    for epoch in range(numEpochs):
        print('Epoch', epoch, 'running...')
        for i, (inputs, labels) in enumerate(train_loader, 0):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

def test_network(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Test accuracy: %d %%' % (100 * correct / total))

train_network(model, train_loader, criterion, optimizer)
test_network(model, test_loader)


Epoch 0 running...
Epoch 1 running...
Epoch 2 running...
Epoch 3 running...
Epoch 4 running...
Epoch 5 running...
Epoch 6 running...
Epoch 7 running...
Epoch 8 running...
Epoch 9 running...
Test accuracy: 36 %
